In [ ]:
import pandas as pd
from pandas_profiling import ProfileReport
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn import tree
from sklearn.metrics import accuracy_score

In [ ]:
def GetDatosEncuesta():
    """Obtener el dataset.

    Devuelve
    --------
        pd.DataFrame: El dataset con los datos de la encuesta descargado desde el drive.
    """
    return pd.read_csv('https://drive.google.com/uc?export=download&id=1i-KJ2lSvM7OQH0Yd59bX01VoZcq8Sglq')

def GetEncuesta():
    """Obtener el dataset.

    Devuelve
    --------
        pd.DataFrame: El dataset con los resultados de la encuesta descargado desde el drive.
    """
    return pd.read_csv('https://drive.google.com/uc?export=download&id=1km-AEIMnWVGqMtK-W28n59hqS5Kufhd0')

# Preguntas inciales
- ¿Los datos están completos? ¿son consistentes?
- ¿Las columnas aportan información relevante?
- ¿Los datos sirven para generar una conclusión?
- ¿Los datos se encuentra balanceados?
- ¿Se pueden eliminar entradas con poca importancia (outliers)?
- ¿Podemos agrupar datos?
- ¿Podemos imputar datos faltantes?


Para responder estas preguntas realizamos el siguiente análisis

# Inspección inicial de datos de la encuesta
Importamos los datos en un dataframe y hacemos un análisis de la estructura del mismo.

In [ ]:
# Importamos csv con los datos
df_datos_encuesta = GetDatosEncuesta()

In [ ]:
# Inspeccionamos las dimensiones iniciales del dataframe
df_datos_encuesta.shape

In [ ]:
# Analizamos la columna 'id_usuario' es busca de usuarios repetidos
df_datos_encuesta['id_usuario'].duplicated(keep=False).value_counts()

In [ ]:
# Verifico que no tenga valores nulos
df_datos_encuesta['id_usuario'].isnull().sum()

# Análisis y transformación de dataframe con resultados
Importamos dataframe con los resultados de la encuesta, realizamos un análisis de los datos y mergeamos el mismo con el dataframe de los datos de la encuesta.

In [ ]:
df_encuesta = GetEncuesta()

In [ ]:
# Analizamos el dataframe df_encuesta, verificando que no tenga valores null
df_encuesta.head()

In [ ]:
df_encuesta.shape

In [ ]:
df_encuesta.isnull().sum()

In [ ]:
# Analizamos la columna 'id_usuario' es busca de usuarios repetidos
df_encuesta['id_usuario'].duplicated(keep=False).value_counts()

In [ ]:
# Mergeamos los dataframe
df = pd.merge(left = df_datos_encuesta, right = df_encuesta, how = 'inner', on = 'id_usuario')

In [ ]:
df.head()

In [ ]:
# DEBUG
df.shape

In [ ]:
# Elimino la columna id_usuario, porque ya no nos aporta informacion
df.drop(columns=['id_usuario'], axis=1, inplace=True)

In [ ]:
# DEBUG
df.head()

# Análisis de columnas
Realizamos un análisis de las columnas del data frame. La estructura utilizada será: preguntas - análisis - conclusión.

# <span style="color:red">Id de ticket</span>

### Preguntas

### Análisis

In [ ]:
df.id_ticket.value_counts()

In [ ]:
df_aux = df.id_ticket.value_counts().reset_index()
df_aux.columns.values[-1] = 'cantidad_espectadores'
df_aux = df_aux[["cantidad_espectadores"]].value_counts().reset_index()
df_aux.columns.values[-1] = 'count'

sns.catplot(data=df_aux, x='cantidad_espectadores', y='count', kind='bar')

plt.title('Cuantos espectadores repiten id de ticket')
plt.ylabel("Cantidad de espectadores")
plt.xlabel("Cantidad con mismo id de ticket")
plt.show()

In [ ]:
# Eliminamos las columna "id_ticket" porque no nos aporta información relavante para la predicción
df.drop(columns= 'id_ticket', inplace=True)

In [ ]:
# DEBUG
df.head()

In [ ]:
# DEBUG
df.shape

### Conclusión

Podemos ver que al agrupar los id_ticket y contar la cantidad de espectadores para cada uno los valores que obtenemos para casi todo el dataset es 1 o 2 espectadores. Por esta razón decidimos eliminar la columna id_ticket ya que para aprender el modelo en este caso se debería memorizar el id_ticket y no es lo que buscamos lograr.

# <span style="color:red">Nombre</span>

### Preguntas

- ¿El dato esta completo en todas las filas?
- ¿Influye el nombre en la columna volveria?

### Análisis

In [ ]:
df['nombre'].isnull().sum()

In [ ]:
df['nombre'].replace({'Señorita ': '', 'Señor ': '', 'Señora ':''}, regex = True, inplace = True) 

In [ ]:
df[['nombre']] = df['nombre'].str.split(pat = " ", n = 1, expand = True)[0]

In [ ]:
# Armo dos nuevos dataframe con los que volverian y los que no
df_vuelven = df[df['volveria'] == 1]
df_no_vuelven = df[df['volveria'] == 0]

In [ ]:
df_vuelven['nombre'].value_counts().head()

In [ ]:
df_no_vuelven['nombre'].value_counts().head()

In [ ]:
print("Vuelven", df_vuelven.shape)
print("No vuelven", df_no_vuelven.shape)

### Conclusión

Como podemos ver de las 307 personas que volverian 7 tienen como nombre Olga, en el caso de las 494 que no volverian 19 tienen como nombre Juan.

# <span style="color:red">Precio de ticket</span>

### Preguntas

- ¿El dato esta completo en todas las filas?
- ¿Los datos tienen mucha varianza?
- ¿Puedo quedarme con los valores mas representativos?

### Análisis

In [ ]:
# Verificamos que no tenga valores nulos
df['precio_ticket'].isnull().sum()

In [ ]:
# Vemos la distribución de los valores
df['precio_ticket'].value_counts()

In [ ]:
plt.figure(dpi=100)
plt.title("Distribución del precio del ticket y\n la decisión de volver")
sns.boxplot(
    data=df,
    y='precio_ticket',
    x='volveria'
)
plt.ylabel("Precio del ticket")
plt.xlabel("Volvería")
plt.xticks([False, True], ["No", "Sí"])
plt.show()

In [ ]:
# Eliminamos los valores mayores a 15
df.drop(df[df['precio_ticket'] > 15].index, inplace=True)

### Conclusión

Como podemos ver la mayoría de los valores representativos de la columna 'precio_ticket' son los menores a 15

# <span style="color:red">Nombre de sede</span>

### Preguntas

- ¿El dato esta completo en todas las filas?
- ¿Los datos tienen mucha varianza?
- ¿Puedo quedarme con los valores mas representativos?
- ¿Puedo completar los valores faltantes?

### Análisis

In [ ]:
# Verificamos que no tenga valores nulos
df['nombre_sede'].isnull().sum()

In [ ]:
# En este caso como vimos que solo tiene 2 filas con valores null las descartamos
bool_nombre_sede = pd.notnull(df['nombre_sede'])  
df = df[bool_nombre_sede]  

In [ ]:
# Vemos la distribución de los valores
df['nombre_sede'].value_counts()

In [ ]:
# Quito 'fiumark' del nombre de la sede, para una mejor visualizacion en los graficos
df['nombre_sede'].replace(to_replace = 'fiumark_quilmes', value ='Quilmes', inplace=True) 
df['nombre_sede'].replace(to_replace = 'fiumark_chacarita', value ='Chacarita', inplace=True) 
df['nombre_sede'].replace(to_replace = 'fiumark_palermo', value ='Palermo', inplace=True) 

In [ ]:
df['nombre_sede'].value_counts().sort_values(ascending=False) \
.plot(title = 'Porcentaje de espectadores por sede', 
      ylabel = '',
      kind='pie', 
      autopct='%1.0f%%')

In [ ]:
# Comparamos la sede a la que fueron y la decisión de volver
plt.figure(dpi=150)
df_aux = df[["volveria", "nombre_sede"]].value_counts().reset_index()
df_aux.columns.values[2] = 'count'
df_aux.volveria.replace(to_replace=[True, False], value=["Sí", "No"], inplace=True)

sns.catplot(data=df_aux, x='nombre_sede', y='count', hue='volveria', kind='bar', legend=False)

plt.legend(title = "Volvería")
plt.ylabel("Cantidad de espectadores")
plt.xlabel("Sede")
plt.title('Relación sede y decisión de volver')
plt.show()

### Conclusión

Como solo tenemos dos valores faltantes los eliminamos.

La mayoría de las personas concurrieron a la sede de Palermo

No podemos extraer un conclusión con el último gráfico. 

# <span style="color:red">Amigos y Parientes</span>

### Preguntas

- ¿El dato esta completo en todas las filas?
- ¿Son consistentes?
- ¿Los datos tienen mucha varianza?
- ¿Puedo completar los valores faltantes?

### Análisis

- Verificamos que no tengan valores nulos y que solo tengan valores numericos enteros positivos

In [ ]:
df['amigos'].isnull().sum()

In [ ]:
df['parientes'].isnull().sum()

- Vemos la distribución de los valores de amigos y parientes

In [ ]:
df['amigos'].value_counts()

In [ ]:
df['parientes'].value_counts()

In [ ]:
# Creamos una nueva columna 'cantidad_acompanantes' que sea la suma de la cantidad de amigos y parientes
df['cantidad_acompanantes'] = df['amigos'] + df['parientes']

In [ ]:
# Elimino las columnas amigos y parientes
df.drop(columns=['amigos', 'parientes'], axis=1, inplace=True)

In [ ]:
# Nos parece relevante si fue acompañado o no, por eso creamos un nueva columna 'fue_acompanado' para falicitar el uso
df['fue_acompanado'] = df['cantidad_acompanantes'].apply(lambda x: 1 if x > 0 else 0)

In [ ]:
ax = df['fue_acompanado'].replace(to_replace=[1, 0], value=['Sí', 'No']).value_counts().sort_values(ascending=False) \
.plot(title = 'Espectadores acompañados vs solos', 
      ylabel = 'Cantidad de espectadores',
      xlabel = 'Fue acompañado',
      kind='bar')

ax.set_xlabel('Cantidad de acompañantes')

In [ ]:
ax = df['cantidad_acompanantes'].plot(title = 'Distribución de la cantidad de acompañantes', kind='hist', bins=20)

ax.set_xlabel('Cantidad de acompañantes')
ax.set_ylabel('Cantidad de espectadores')

### Conclusión

La mayoría de las personas no van acompanadas al cine.

# <span style="color:red">Edad</span>

### Preguntas

- ¿Tiene valores completos?
- ¿Puedo completar los valores faltantes?

### Análisis

In [ ]:
# Analizamos la columna 'edad' 
df['edad'].isnull().sum()

In [ ]:
# Eliminamos columnas nulas
bool_tiene_edad = pd.notnull(df['edad'])  
df = df[bool_tiene_edad]  

In [ ]:
# Redondeo los valores
df['edad'] = df['edad'].round(0).astype(int)

### Conclusión

La columna edad tiene un faltante de 160 filas, la variable "edad" es de tipo MNAR (es dependiente de los datos no observados).

Al no tener una forma de imputarla a partir de las variables observadas decidimos eliminar las filas con edad nula


# <span style="color:red">Género</span> 

### Preguntas

- ¿Tiene valores completos?
- ¿Son consistentes?
- ¿Lo valores están distribuidos?
- ¿La distribución aporta relevancia para establecer un comportamiento en la decisión de volver?

### Análisis

In [ ]:
# Verifico que la columna genero no tenga nulos y solo tenga valores mujer/hombre
df['genero'].isnull().sum()

In [ ]:
df['genero'].value_counts()

In [ ]:
# Vemos en porcentaje el género de los espectadores
plt.figure(dpi=85)
df.genero.value_counts().plot(kind='pie', autopct='%1.0f%%', ylabel='Género')

In [ ]:
# Comparamos el género y la decisión de volver
plt.figure(dpi=150)
df_aux = df[["volveria", "genero"]].value_counts().reset_index()
df_aux.columns.values[2] = 'count'
df_aux.volveria.replace(to_replace=[True, False], value=["Sí", "No"], inplace=True)

sns.catplot(data=df_aux, x='genero', y='count', hue='volveria', kind='bar', legend=False)

plt.title('Relación género y decisión de volver')
plt.legend(title = "Volvería")
plt.ylabel("Cantidad de espectadores")
plt.xlabel("Género")
plt.show()


### Conclusión

Podemos ver que en su mayoría los espectadores son hombres.

Podemos apreciar que incluso mientras que la cantidad de hombres que participaron es mayor que la cantidad de mujeres, es significativamente mayor la cantidad de mujeres que decidirían volver comparada con la cantidad de hombres que elegirían volver.

# <span style="color:red">Tipo de sala</span>

### Preguntas

- ¿Tiene valores completos?
- ¿Son consistentes?
- ¿Lo valores están distribuidos?
- ¿La distribución aporta relevancia para establecer un comportamiento en la decisión de volver?

### Análisis

In [ ]:
# Analizamos la columna 'tipo_de_sala'
# Verificamos que no tenga valores nulos
df['tipo_de_sala'].isnull().sum()

In [ ]:
df['tipo_de_sala'].value_counts()

In [ ]:
# Reemplazamos normal por 2d
df['tipo_de_sala'].replace(to_replace='normal', value='2d', inplace=True)

In [ ]:
# DEBUG
df['tipo_de_sala'].value_counts()

In [ ]:
df['tipo_de_sala'].value_counts().sort_values(ascending=False) \
.plot(title = 'Cantidad de espectadores por tipo de sala', 
      xlabel = 'Tipo de sala', 
      ylabel = 'Cantidad de espectadores',
      kind='bar')

In [ ]:
# Comparamos la sala a la que fueron y la decisión de volver
plt.figure(dpi=150)
df_aux = df[["volveria", "tipo_de_sala"]].value_counts().reset_index()
df_aux.columns.values[2] = 'count'
df_aux.volveria.replace(to_replace=[True, False], value=["Sí", "No"], inplace=True)

sns.catplot(data=df_aux, x='tipo_de_sala', y='count', hue='volveria', kind='bar', 
                legend=False, order=['2d', '3d', '4d'])

plt.legend(title = "Volvería")
plt.ylabel("Cantidad de espectadores")
plt.xlabel("Tipo de sala")
plt.title('Relación tipo de sala y decisión de volver')
plt.show()

### Conclusión

Graficamente vemos que la mayoría de los encuestados concurrieron a salas 4D

Podemos observar que en el caso de las salas:
- 2D es superior la cantidad de personas que decidirían volver respecto de las que no.
- 3D no se aprecia una diferencia significativa en la decisión de volver.
- 4D es considerable la cantidad de personas que no volverían comparada con las que sí.

Además observamos que si bien la cantidad de espectadores de las salas 2d es inferior a la de 4d, la cantidad de personas que decidiría volver es superior.

# Justificar la eliminacion de las columnas

nombre

fila (no tocar, hasta el jueves o viernes a la tarde)

id_ticket

-------------------------------------------------------------------

In [ ]:
# Analizamos la columna 'fila'
df['fila'].isnull().sum()

# Transformación de datos de la encuesta
Aplicamos los cambios necesarios para realizar una limpieza de los datos, basandonos en el análisis inicial.

In [ ]:
# Como la mayoria de los valores de la columna fila son nulos decidimos eliminarla
df.drop(columns='fila', inplace=True)

# Gráficos

In [ ]:
# Vemos en porcentaje si las personas volverían o no
plt.figure(dpi=85)
df.volveria.replace(to_replace=[True, False], value=["Sí", "No"]).value_counts().plot(kind='pie', autopct='%1.0f%%', ylabel='¿Volvería?')

Podemos apreciar que la mayoría de los espectadores no volverían

In [ ]:
# Vemos cuales fueron las salas más elegidas según las sedes
plt.figure(dpi=100)

df_aux = df[["tipo_de_sala", "nombre_sede"]].value_counts().reset_index()
df_aux.columns.values[2] = 'count'

df_aux = df_aux.pivot("tipo_de_sala", "nombre_sede", "count")

g = sns.heatmap(df_aux, square=True, cmap="Wistia", xticklabels=True)

g.set_xlabel("Nombre de sede")
g.set_ylabel("Tipo de sala")
g.set_title("Relación sede y tipo de sala elegidas")
  
plt.show()

Podemos ver que para todos los tipos de salas la sede más elegida fue Palermo y que la sala 4d de Palermo fue la más elegida de todas.

In [ ]:
df_aux = df[["volveria", "genero", "tipo_de_sala", "nombre_sede"]].value_counts().reset_index()
df_aux.columns.values[-1] = 'cantidad_espectadores'
df_aux.volveria.replace(to_replace=[True, False], value=["sí", "no"], inplace=True)

In [ ]:
df_hombre = df_aux[df_aux["genero"] == "hombre"]

plt.figure()

g = sns.catplot(data=df_hombre, x='tipo_de_sala', y='cantidad_espectadores', hue='nombre_sede', kind='bar',
            order=['2d', '3d', '4d'], col='volveria')

(g.set_axis_labels("Tipo de sala", "Cantidad de espectadores")
  .set_titles("Distribución de hombres que {col_name} {col_var} por sala y sede"))  

plt.show()

In [ ]:
df_mujer = df_aux[df_aux["genero"] == "mujer"]

plt.figure()
g = sns.catplot(data=df_mujer, x='tipo_de_sala', y='cantidad_espectadores', hue='nombre_sede', kind='bar',
            order=['2d', '3d', '4d'], col ='volveria')

(g.set_axis_labels("Tipo de sala", "Cantidad de espectadores")
  .set_titles("Distribución de mujeres que {col_name} {col_var} por sala y sede")
)  

plt.show()

# Conclusiones

- Con el gráfico boxplot podemos ver que la mayoría de los valores representativos de la columna 'precio_ticket' es menor a 15 
 
  Hablar con mica porque no se incluyo en feature engineering
  
  

- Con los últimos dos gráficos llegamos a las siguientes conclusiones
    - En el caso de los hombres para todas las sedes y salas la mayoría no volvería. 
    - En el caso de las mujeres 
        - Para las salas 2d y 3d se puede ver que en todas las sedes la mayoría volvería.
        - Para las salas 4d
            - En el caso de la sede de Chacarita la mayoría volvería
            - En el caso de la sede de Quilmes no se tiene suficiente información para establecer un comportamiento
            - En el caso de la sede de Palermo podemos notar que la mayoría no volvería
          

# Feature Engineering

In [ ]:
def FeatureEngineering(df):
    
    df_clean = df.copy()
    
    df_clean.drop(columns=['nombre', 'id_ticket', 'id_usuario'], inplace = True)
    df_clean['cantidad_acompanantes'] = df_clean['amigos'] + df_clean['parientes']
    df_clean.drop(columns=['amigos', 'parientes'], axis=1, inplace = True)
    df_clean['fue_acompanado'] = df_clean['cantidad_acompanantes'].apply(lambda x: 1 if x > 0 else 0)
    df_clean['tipo_de_sala'].replace(to_replace = 'normal', value = '2d', inplace = True)
    df_clean['nombre_sede'].replace(to_replace = 'fiumark_quilmes', value ='Quilmes', inplace=True) 
    df_clean['nombre_sede'].replace(to_replace = 'fiumark_chacarita', value ='Chacarita', inplace=True) 
    df_clean['nombre_sede'].replace(to_replace = 'fiumark_palermo', value ='Palermo', inplace=True) 
    df_clean.drop(columns = 'fila', inplace = True)
            
    return df_clean

In [ ]:
def Predict(df):
    pred = []
    values = df.values
    for val in values:
        if val[1] == 'hombre':
            pred.append(0)
        else:
            if (val[0] == '4d' and val[4] == 'Palermo'):
                pred.append(0)
            else:
                pred.append(1)
     
    return pred

In [ ]:
def baseline(df):
    """    
    Input: dataframe
    Output: lista con las predicciones    
    """    
    # Aplicamos feature engineering a los datos de entrada
    entrada = FeatureEngineering(df)
    pred = Predict(entrada)
    
    return pred

In [ ]:
# Generamos dataframe para probar
df_datos_test = GetDatosEncuesta()

df_volveria_test = GetEncuesta()
df_volveria_test = df_volveria_test.volveria


In [ ]:
predicciones = baseline(df_datos_test)
accuracy_score(df_volveria_test, predicciones)

# Comentarios:

	* Razon por la que no importa a la prediccion (apoyarse con graficos)
		nombre, fila (sesgo)
	
	* Chequear el motivo por el cual faltan
	
	
* Graficos
    - Ordenar graficos
    - Medir porcentajes en base a gente encuestada
    - Agregar conclusion al informe de porque no aporta informacion
		
		
* Coherencia entre entrenamiento y prediccion	


* Preguntas de negocio - analisis - conclusion



## Re-entrega: antes del 17 de noviembre